In [1]:
######### libraries ################
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
from scipy import stats
from scipy.signal import hilbert
from sklearn import preprocessing, linear_model
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM

Using TensorFlow backend.


In [2]:
############ declaring functions #############
def scale_plot_size(factor=3.0):
    default_figsize = mpl.rcParamsDefault['figure.figsize']
    mpl.rcParams['figure.figsize'] = [val*factor for val in default_figsize]
scale_plot_size(1.5)

def rms(df):
    return np.sqrt(df.dot(df)/df.size)


In [3]:
######### reading file ##########
header = ["Hr", "Min","Sec","uSec", "X","Y"]

training_set_path = os.path.expanduser('~/Vibrational_analysis/data/IEEE_dataset/Learning_set/Bearing2_1')

dirList = os.listdir(os.path.expanduser(training_set_path))
dirList.sort()
n = 0

for sets in dirList: 
    if sets == 'temp_00001.csv':
        break
    df = pd.read_csv(training_set_path+"/"+sets, header=None)
    df.columns = header
    data = np.array(df,np.float64)
    columnx = data[:,4]
    columny = data[:,5]

    if n==0 :
        x = columnx
        y = columny

        kurtx = stats.kurtosis(columnx)
        kurty = stats.kurtosis(columny)

        rmsx = rms(columnx)
        rmsy = rms(columny)
        
        n = 1
    else:
        x= np.append(x,columnx)
        y= np.append(y,columny)

        kurtx = np.append(kurtx,stats.kurtosis(columnx))
        kurty = np.append(kurty,stats.kurtosis(columny))

        rmsx = np.append(rmsx,rms(columnx))
        rmsy = np.append(rmsy,rms(columny))


In [4]:
########### defining input and target variables ###############
X = np.vstack ((kurtx,kurty,rmsx,rmsy))
threshold = int(kurtx.shape[0]*0.25)
output = np.arange (kurtx.shape[0]-1,-1,-1,dtype=np.float64)
output = np.reshape(output,(kurtx.shape[0],1))
Y = np.where( output>threshold , 0 , 1 )
Xnm = preprocessing.normalize(X)
Xnm = Xnm.transpose()

X_train, X_test, Y_train, Y_test = train_test_split(Xnm, Y, test_size=0.2, random_state=69)

In [5]:
################ create model ##################
pred = Sequential()
pred.add(Dense(30,input_dim = 4,init = 'normal',activation = 'relu'))
# pred.add(Dropout(0.5))
pred.add(Dense(20, init = 'normal',activation = 'relu'))
# pred.add(Dropout(0.5))
pred.add(Dense(20, init = 'normal',activation = 'relu'))
# pred.add(Dropout(0.5))
pred.add(Dense(20, init = 'normal',activation = 'relu'))
# pred.add(Dropout(0.5))
pred.add(Dense(20, init = 'normal',activation = 'relu'))
# pred.add(Dropout(0.5))
pred.add(Dense(1,activation = 'sigmoid'))

############### compile model #############
pred.compile(loss = 'binary_crossentropy',optimizer = 'adam', metrics=['accuracy','binary_accuracy'])

############### fit model #################
pred.fit(X_train,Y_train,epochs = 500,validation_split = 0.3,batch_size = 32,verbose =1)

############### predict model #############
predicted = pred.predict(X_test)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(30, activation="relu", kernel_initializer="normal", input_dim=4)`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(20, activation="relu", kernel_initializer="normal")`
  """
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(20, activation="relu", kernel_initializer="normal")`
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(20, activation="relu", kernel_initializer="normal")`
  if __name__ == '__main__':
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense

Train on 445 samples, validate on 192 samples
Epoch 1/500
445/445 [==============================] - 0s 382us/step - loss: 0.6900 - acc: 0.7146 - binary_accuracy: 0.7146 - val_loss: 0.6876 - val_acc: 0.6979 - val_binary_accuracy: 0.6979
Epoch 2/500
445/445 [==============================] - 0s 57us/step - loss: 0.6818 - acc: 0.7640 - binary_accuracy: 0.7640 - val_loss: 0.6812 - val_acc: 0.6979 - val_binary_accuracy: 0.6979
Epoch 3/500
445/445 [==============================] - 0s 66us/step - loss: 0.6722 - acc: 0.7640 - binary_accuracy: 0.7640 - val_loss: 0.6736 - val_acc: 0.6979 - val_binary_accuracy: 0.6979
Epoch 4/500
445/445 [==============================] - 0s 62us/step - loss: 0.6605 - acc: 0.7640 - binary_accuracy: 0.7640 - val_loss: 0.6639 - val_acc: 0.6979 - val_binary_accuracy: 0.6979
Epoch 5/500
445/445 [==============================] - 0s 63us/step - loss: 0.6448 - acc: 0.7640 - binary_accuracy: 0.7640 - val_loss: 0.6513 - val_acc: 0.6979 - val_binary_accuracy: 0.6979
Epo

Epoch 44/500
445/445 [==============================] - 0s 71us/step - loss: 0.2989 - acc: 0.8517 - binary_accuracy: 0.8517 - val_loss: 0.3096 - val_acc: 0.8542 - val_binary_accuracy: 0.8542
Epoch 45/500
445/445 [==============================] - 0s 61us/step - loss: 0.2985 - acc: 0.8629 - binary_accuracy: 0.8629 - val_loss: 0.3272 - val_acc: 0.8594 - val_binary_accuracy: 0.8594
Epoch 46/500
445/445 [==============================] - 0s 61us/step - loss: 0.2997 - acc: 0.8517 - binary_accuracy: 0.8517 - val_loss: 0.3318 - val_acc: 0.8490 - val_binary_accuracy: 0.8490
Epoch 47/500
445/445 [==============================] - 0s 64us/step - loss: 0.3136 - acc: 0.8360 - binary_accuracy: 0.8360 - val_loss: 0.3214 - val_acc: 0.8385 - val_binary_accuracy: 0.8385
Epoch 48/500
445/445 [==============================] - 0s 63us/step - loss: 0.3146 - acc: 0.8449 - binary_accuracy: 0.8449 - val_loss: 0.3303 - val_acc: 0.8542 - val_binary_accuracy: 0.8542
Epoch 49/500
445/445 [=======================

Epoch 87/500
445/445 [==============================] - 0s 77us/step - loss: 0.2743 - acc: 0.8719 - binary_accuracy: 0.8719 - val_loss: 0.3019 - val_acc: 0.8750 - val_binary_accuracy: 0.8750
Epoch 88/500
445/445 [==============================] - 0s 61us/step - loss: 0.2706 - acc: 0.8742 - binary_accuracy: 0.8742 - val_loss: 0.2842 - val_acc: 0.8750 - val_binary_accuracy: 0.8750
Epoch 89/500
445/445 [==============================] - 0s 64us/step - loss: 0.2647 - acc: 0.8719 - binary_accuracy: 0.8719 - val_loss: 0.2906 - val_acc: 0.8646 - val_binary_accuracy: 0.8646
Epoch 90/500
445/445 [==============================] - 0s 57us/step - loss: 0.2619 - acc: 0.8764 - binary_accuracy: 0.8764 - val_loss: 0.2847 - val_acc: 0.8698 - val_binary_accuracy: 0.8698
Epoch 91/500
445/445 [==============================] - 0s 61us/step - loss: 0.2625 - acc: 0.8719 - binary_accuracy: 0.8719 - val_loss: 0.2846 - val_acc: 0.8698 - val_binary_accuracy: 0.8698
Epoch 92/500
445/445 [=======================

Epoch 130/500
445/445 [==============================] - 0s 65us/step - loss: 0.2294 - acc: 0.9079 - binary_accuracy: 0.9079 - val_loss: 0.2538 - val_acc: 0.8906 - val_binary_accuracy: 0.8906
Epoch 131/500
445/445 [==============================] - 0s 55us/step - loss: 0.2306 - acc: 0.8944 - binary_accuracy: 0.8944 - val_loss: 0.2481 - val_acc: 0.8854 - val_binary_accuracy: 0.8854
Epoch 132/500
445/445 [==============================] - 0s 70us/step - loss: 0.2823 - acc: 0.8674 - binary_accuracy: 0.8674 - val_loss: 0.2474 - val_acc: 0.8854 - val_binary_accuracy: 0.8854
Epoch 133/500
445/445 [==============================] - 0s 61us/step - loss: 0.2557 - acc: 0.8787 - binary_accuracy: 0.8787 - val_loss: 0.2625 - val_acc: 0.8906 - val_binary_accuracy: 0.8906
Epoch 134/500
445/445 [==============================] - 0s 54us/step - loss: 0.2367 - acc: 0.8921 - binary_accuracy: 0.8921 - val_loss: 0.2767 - val_acc: 0.8906 - val_binary_accuracy: 0.8906
Epoch 135/500
445/445 [=================

...

Epoch 302/500
445/445 [==============================] - 0s 55us/step - loss: 0.1609 - acc: 0.9348 - binary_accuracy: 0.9348 - val_loss: 0.1970 - val_acc: 0.9427 - val_binary_accuracy: 0.9427
Epoch 303/500
445/445 [==============================] - 0s 57us/step - loss: 0.1904 - acc: 0.9146 - binary_accuracy: 0.9146 - val_loss: 0.2073 - val_acc: 0.9323 - val_binary_accuracy: 0.9323
Epoch 304/500
445/445 [==============================] - 0s 54us/step - loss: 0.1789 - acc: 0.9191 - binary_accuracy: 0.9191 - val_loss: 0.2123 - val_acc: 0.9010 - val_binary_accuracy: 0.9010
Epoch 305/500
445/445 [==============================] - 0s 60us/step - loss: 0.2044 - acc: 0.9169 - binary_accuracy: 0.9169 - val_loss: 0.2589 - val_acc: 0.8802 - val_binary_accuracy: 0.8802
Epoch 306/500
445/445 [==============================] - 0s 55us/step - loss: 0.2107 - acc: 0.8989 - binary_accuracy: 0.8989 - val_loss: 0.1671 - val_acc: 0.9375 - val_binary_accuracy: 0.9375
Epoch 307/500
445/445 [=================

Epoch 345/500
445/445 [==============================] - 0s 75us/step - loss: 0.1846 - acc: 0.9191 - binary_accuracy: 0.9191 - val_loss: 0.1779 - val_acc: 0.9323 - val_binary_accuracy: 0.9323
Epoch 346/500
445/445 [==============================] - 0s 57us/step - loss: 0.1907 - acc: 0.9236 - binary_accuracy: 0.9236 - val_loss: 0.1700 - val_acc: 0.9375 - val_binary_accuracy: 0.9375
Epoch 347/500
445/445 [==============================] - 0s 55us/step - loss: 0.2071 - acc: 0.9079 - binary_accuracy: 0.9079 - val_loss: 0.1862 - val_acc: 0.9219 - val_binary_accuracy: 0.9219
Epoch 348/500
445/445 [==============================] - 0s 55us/step - loss: 0.1817 - acc: 0.9191 - binary_accuracy: 0.9191 - val_loss: 0.1661 - val_acc: 0.9375 - val_binary_accuracy: 0.9375
Epoch 349/500
445/445 [==============================] - 0s 59us/step - loss: 0.1702 - acc: 0.9393 - binary_accuracy: 0.9393 - val_loss: 0.1654 - val_acc: 0.9323 - val_binary_accuracy: 0.9323
Epoch 350/500
445/445 [=================

Epoch 388/500
445/445 [==============================] - 0s 70us/step - loss: 0.1820 - acc: 0.9303 - binary_accuracy: 0.9303 - val_loss: 0.2026 - val_acc: 0.9427 - val_binary_accuracy: 0.9427
Epoch 389/500
445/445 [==============================] - 0s 56us/step - loss: 0.2210 - acc: 0.8876 - binary_accuracy: 0.8876 - val_loss: 0.2184 - val_acc: 0.9375 - val_binary_accuracy: 0.9375
Epoch 390/500
445/445 [==============================] - 0s 54us/step - loss: 0.1571 - acc: 0.9348 - binary_accuracy: 0.9348 - val_loss: 0.1623 - val_acc: 0.9427 - val_binary_accuracy: 0.9427
Epoch 391/500
445/445 [==============================] - 0s 61us/step - loss: 0.1504 - acc: 0.9393 - binary_accuracy: 0.9393 - val_loss: 0.1950 - val_acc: 0.9375 - val_binary_accuracy: 0.9375
Epoch 392/500
445/445 [==============================] - 0s 64us/step - loss: 0.1590 - acc: 0.9393 - binary_accuracy: 0.9393 - val_loss: 0.2077 - val_acc: 0.9427 - val_binary_accuracy: 0.9427
Epoch 393/500
445/445 [=================

Epoch 431/500
445/445 [==============================] - 0s 63us/step - loss: 0.1629 - acc: 0.9326 - binary_accuracy: 0.9326 - val_loss: 0.1586 - val_acc: 0.9375 - val_binary_accuracy: 0.9375
Epoch 432/500
445/445 [==============================] - 0s 58us/step - loss: 0.1559 - acc: 0.9371 - binary_accuracy: 0.9371 - val_loss: 0.1571 - val_acc: 0.9375 - val_binary_accuracy: 0.9375
Epoch 433/500
445/445 [==============================] - 0s 52us/step - loss: 0.1509 - acc: 0.9393 - binary_accuracy: 0.9393 - val_loss: 0.1562 - val_acc: 0.9375 - val_binary_accuracy: 0.9375
Epoch 434/500
445/445 [==============================] - 0s 55us/step - loss: 0.1779 - acc: 0.9281 - binary_accuracy: 0.9281 - val_loss: 0.1832 - val_acc: 0.9219 - val_binary_accuracy: 0.9219
Epoch 435/500
445/445 [==============================] - 0s 60us/step - loss: 0.1728 - acc: 0.9191 - binary_accuracy: 0.9191 - val_loss: 0.1570 - val_acc: 0.9375 - val_binary_accuracy: 0.9375
Epoch 436/500
445/445 [=================

Epoch 474/500
445/445 [==============================] - 0s 57us/step - loss: 0.1818 - acc: 0.9079 - binary_accuracy: 0.9079 - val_loss: 0.1581 - val_acc: 0.9375 - val_binary_accuracy: 0.9375
Epoch 475/500
445/445 [==============================] - 0s 54us/step - loss: 0.1566 - acc: 0.9348 - binary_accuracy: 0.9348 - val_loss: 0.1651 - val_acc: 0.9375 - val_binary_accuracy: 0.9375
Epoch 476/500
445/445 [==============================] - 0s 53us/step - loss: 0.1501 - acc: 0.9326 - binary_accuracy: 0.9326 - val_loss: 0.1579 - val_acc: 0.9427 - val_binary_accuracy: 0.9427
Epoch 477/500
445/445 [==============================] - 0s 55us/step - loss: 0.1455 - acc: 0.9371 - binary_accuracy: 0.9371 - val_loss: 0.1950 - val_acc: 0.9479 - val_binary_accuracy: 0.9479
Epoch 478/500
445/445 [==============================] - 0s 54us/step - loss: 0.1598 - acc: 0.9258 - binary_accuracy: 0.9258 - val_loss: 0.1550 - val_acc: 0.9427 - val_binary_accuracy: 0.9427
Epoch 479/500
445/445 [=================